# 21번째 제출 

### 패치 내용 
gray 스케일 -> rgb 적용 스케일 

In [1]:
import pandas as pd
import numpy as np

from sklearn.svm import SVC
from sklearn.decomposition import PCA
import lightgbm as lgb
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier

import os
from tqdm import tqdm
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler



In [11]:
training_data = pd.read_csv('C:/Users/유재림/Desktop/ML23_2-main/archive/fashion-mnist_train.csv')
train_y = training_data['label'] # 라벨값 

# train_X = training_data.drop('label',axis=1) -> 기존 X train 데이터 

In [5]:
# 이미지 픽셀값 불러오기
new_dataX = np.load('rgb_images.npy') # new_dataX 데이터 셋은 이미 데이터 정규화가 되어 있는 상태 

num_images = len(new_dataX)
print(num_images)
# new_dataX

60000


In [7]:
# 4D 배열 2D로 펼치기 
flat_images = new_dataX.reshape((new_dataX.shape[0], -1))

# Convert to DataFrame
columns = [f'pixel_{i}' for i in range(flat_images.shape[1])]
RGB_X = pd.DataFrame(data=flat_images, columns=columns)

# Display the DataFrame - 이건 그냥 판다스로 내부 구조 파악해보기 
RGB_X

,pixel_0,pixel_1,pixel_2,pixel_3,pixel_4,pixel_5,pixel_6,pixel_7,pixel_8,pixel_9,...,pixel_3126,pixel_3127,pixel_3128,pixel_3129,pixel_3130,pixel_3131,pixel_3132,pixel_3133,pixel_3134,pixel_3135
0,0.267004,0.004874,0.329415,1.0,0.267004,0.004874,0.329415,1.0,0.267004,0.004874,...,0.329415,1.0,0.267004,0.004874,0.329415,1.0,0.267004,0.004874,0.329415,1.0
1,0.267004,0.004874,0.329415,1.0,0.267004,0.004874,0.329415,1.0,0.267004,0.004874,...,0.329415,1.0,0.267004,0.004874,0.329415,1.0,0.267004,0.004874,0.329415,1.0
2,0.267004,0.004874,0.329415,1.0,0.267004,0.004874,0.329415,1.0,0.267004,0.004874,...,0.329415,1.0,0.267004,0.004874,0.329415,1.0,0.267004,0.004874,0.329415,1.0
3,0.267004,0.004874,0.329415,1.0,0.267004,0.004874,0.329415,1.0,0.267004,0.004874,...,0.329415,1.0,0.267004,0.004874,0.329415,1.0,0.267004,0.004874,0.329415,1.0
4,0.267004,0.004874,0.329415,1.0,0.267004,0.004874,0.329415,1.0,0.267004,0.004874,...,0.329415,1.0,0.267004,0.004874,0.329415,1.0,0.267004,0.004874,0.329415,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,0.267004,0.004874,0.329415,1.0,0.267004,0.004874,0.329415,1.0,0.267004,0.004874,...,0.329415,1.0,0.267004,0.004874,0.329415,1.0,0.267004,0.004874,0.329415,1.0
59996,0.267004,0.004874,0.329415,1.0,0.267004,0.004874,0.329415,1.0,0.267004,0.004874,...,0.329415,1.0,0.267004,0.004874,0.329415,1.0,0.267004,0.004874,0.329415,1.0
59997,0.267004,0.004874,0.329415,1.0,0.267004,0.004874,0.329415,1.0,0.267004,0.004874,...,0.329415,1.0,0.267004,0.004874,0.329415,1.0,0.267004,0.004874,0.329415,1.0
59998,0.267004,0.004874,0.329415,1.0,0.267004,0.004874,0.329415,1.0,0.267004,0.004874,...,0.329415,1.0,0.267004,0.004874,0.329415,1.0,0.267004,0.004874,0.329415,1.0


In [12]:
datagen = ImageDataGenerator(
    width_shift_range=0.10,
    height_shift_range=0.10
)

np.random.seed(42)

# Data augmentation and adding to the training set loop
aug_train_X = []
aug_train_y = []

for index, row in tqdm(RGB_X.iterrows(), total=len(RGB_X)):
    random_num = np.random.random()

    img = row.values.reshape((28, 28, 4))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)

    # Original data
    aug_train_X.append(img_array[0])
    aug_train_y.append(train_y[index])

    # Augmented data with 2/3 probability
    
    augmented_img_array = next(datagen.flow(img_array, batch_size=1))
    augmented_img_array = augmented_img_array.squeeze(axis=0)
    aug_train_X.append(augmented_img_array)
    aug_train_y.append(train_y[index])

print(len(aug_train_X))
print(len(aug_train_y))


100%|██████████| 60000/60000 [00:38<00:00, 1553.69it/s]

120000
120000


In [13]:
for index, row in tqdm(RGB_X.iterrows(), total=len(RGB_X)):
    random_num = np.random.random()

    img = row.values.reshape((28, 28, 4))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)

    # Original data
    # aug_train_X.append(img_array[0])
    # aug_train_y.append(train_y[index])

    augmented_img_array = next(datagen.flow(img_array, batch_size=1))
    augmented_img_array = augmented_img_array.squeeze(axis=0)
    aug_train_X.append(augmented_img_array)
    aug_train_y.append(train_y[index])
    
# Convert the lists to numpy arrays
aug_train_X = np.array(aug_train_X)
aug_train_y = np.array(aug_train_y)

# Check the shape of the new arrays
print("Shape of aug_train_X:", aug_train_X.shape)
print("Shape of aug_train_y:", aug_train_y.shape)

100%|██████████| 60000/60000 [00:35<00:00, 1713.50it/s]


Shape of aug_train_X: (180000, 28, 28, 4)
Shape of aug_train_y: (180000,)


In [14]:
aug_train_X = aug_train_X.reshape((aug_train_X.shape[0], -1))
# aug_train_X/=255.0 # RGB 데이터 셋은 이미 정규화가 된 상태로 존재 오히려 /255 하면 오류가 남 

In [15]:
print(aug_train_X.shape)

(180000, 3136)


# 새로운 PCA값 찾기 

In [18]:
from sklearn.decomposition import PCA

view_pca = PCA()
view_pca.fit(aug_train_X)  # X는 데이터

PCA()

In [19]:
# 누적 설명된 분산이 목표치(예: 0.95)를 넘을 때까지 n_components를 증가시킴

# n_components를 실수로 인자를 넣게 되면 요 코드가 먼저 실행되서 best차원 값을 자동으로 넣어줌 
explained_variance_ratio_cumsum = view_pca.explained_variance_ratio_.cumsum()
best_n_components = (explained_variance_ratio_cumsum >= 0.98).argmax() + 1

print(best_n_components)

443


In [16]:
real_pca = PCA( n_components= 0.975 )
aug_train_X = real_pca.fit_transform(aug_train_X)

In [20]:
aug_train_X.shape # 몇 차원까지 줄었는가? -> 분산 0.975일때 669차원까지 

(180000, 669)

In [10]:
# 실험을 위해 이진분류는 제외하고 실험함 

# # 라벨 6에 대한 이진 분류기를 위한 라벨 생성
# binary_train_y = (aug_train_y == 6).astype(int)

# # SVC 이진 분류기 학습
# binary_svc = SVC(gamma='scale', kernel='rbf', C=16)
# binary_svc.fit(aug_train_X, binary_train_y)

SVC(C=16)

In [21]:
# 원래의 분류기 학습
svc = SVC(gamma='scale', kernel='rbf', C=16)
svc.fit(aug_train_X, aug_train_y)

# 테스트 데이터 셋 color이미지 출력 

In [12]:
import matplotlib.pyplot as plt

test_data_folder = "C:/Users/유재림/Desktop/ML23_2-main/ML23_2/submitcode/data"

# 테스트 데이터 로드 및 전처리
test_X = []
file_names = []

print(len(os.listdir(test_data_folder)))

# tqdm으로 래핑
for file_name in tqdm(os.listdir(test_data_folder), desc="Loading and preprocessing"):
    if file_name.endswith(".png"):
        file_path = str(test_data_folder) + "/" + str(file_name)
        try:

            img_array = np.fromfile(file_path, np.uint8)
            img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
            image_array = img / 255.0  # 0부터 1사이의 값
            test_X.append(image_array.flatten())  # 2D 배열을 1D로 펼침

            plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))  # BGR에서 RGB로 변환
            plt.show()

            file_names.append(file_name)
        except Exception as e:
            print(f"Error processing file {file_name}: {str(e)}")


# NumPy 배열로 변환
test_X = np.array(test_X)
print(test_X.shape)

if len(test_X) == 0:
    print("No valid test images found.")
else:
    print("pca 적용")
    test_X = real_pca.transform(test_X)
    print(test_X.shape) # 테스트 PCA shape 

    # 테스트 데이터에 대한 예측
    # binary_predictions = binary_svc.predict(test_X)
    predictions = svc.predict(test_X)

    # 두 분류기의 결과를 합침
    # final_predictions = np.where(binary_predictions == 1, 6, original_predictions)


with open('C:/Users/유재림/Desktop/ML23_2-main/ML23_2/submitcode/testResult2.txt', 'w') as file:
    for i, pred in enumerate(predictions):
        file.write(f"{i:05d} {pred}\n")

10000


Loading and preprocessing: 100%|██████████| 10000/10000 [01:47<00:00, 92.98it/s]


(10000, 784)
pca 적용
(10000, 256)


In [13]:
import sys
import numpy as np
from sklearn.metrics import auc
from collections import Counter

testResult_path = "C:/Users/유재림/Desktop/ML23_2-main/ML23_2/submitcode/testResult2.txt"
label_path = "C:/Users/유재림/Desktop/ML23_2-main/ML23_2/submitcode/label.txt"

# pred에 해당하는 testResult.txt 파일 읽어오는 부분입니다.
with open(testResult_path, 'r') as file1:
    preds = file1.readlines()

# 정답에 해당하는 label.txt 파일 읽어오는 부분입니다.
with open(label_path, 'r') as file2:
    labels = file2.readlines()
    

# pred와 label의 클래스값만 리스트로 변환하는 부분입니다.
p = np.array([pred.strip().split()[1] for pred in preds])
l = np.array([label.strip().split()[1] for label in labels])

# pred의 클래스 개수를 count하는 부분입니다.
predict_label_count_dict = Counter(p)
predict_label_count_dict = dict(sorted(predict_label_count_dict.items()))

## mAP 계산하는 부분입니다.
AP = []
num_class = 10

# 모든 클래스에 대해 반복
for c, freq in predict_label_count_dict.items() :
    TP = 0
    FN = 0

    temp_precision = []
    temp_recall = []
    
    for i in range(len(p)):
        # TP, FN 계산
        if l[i] == c and p[i] == c :
            TP += 1
        elif l[i] != c and p[i] == c :
            FN += 1
        
        # preciison, recall 계산            
        if TP+FN != 0: 
            temp_precision.append(TP/(TP+FN))
            temp_recall.append(TP/freq)

    # AP 배열에 클래스 각각의 AP value 저장
    # auc : preciison-recall curve의 면적 구해줌
    AP.append(auc(temp_recall, temp_precision))
    
mAP = sum(AP) / num_class

# 각각의 클래스에 대한 AP와 mAP의 Table 출력 부분입니다.
class_name = ['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']
table = "| {:<13} | {:<13} |\n".format("Class", "AP") + "|---------------|---------------|\n"

for c_name, ap in zip(class_name, AP):
    table += "| {:<13} | {:<13.3f} |\n".format(c_name, ap)

table += "| {:<13} | {:<13.3f} |\n".format("mAP", mAP)

print(table)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/julie/Machine_Learning/project/teamproject/mAP/mAP/testResult.txt'